Reference:
http://cs230.stanford.edu/projects_winter_2021/reports/70667451.pdf

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, MaxPooling1D, Flatten
%store -r dataset
dataset.head()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/IPython/extensions/storemagic.py:148: UserWarning: This is now an optional IPython functionality, using autorestore/dataset requires you to install the `pickleshare` library.
  obj = db["autorestore/" + arg]


AttributeError: 'NoneType' object has no attribute 'head'

Data from file Data&normalize

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(dataset)

# Create sequences


def create_sequences(data, seq_length):
    sequences = []
    labels = []
    for i in range(len(data) - seq_length):
        sequences.append(data[i:i + seq_length])
        labels.append(data[i + seq_length])
    return np.array(sequences), np.array(labels)


seq_length = 30
X, y = create_sequences(scaled_data, seq_length)

# Reshape X for CNN input
X_cnn = X.reshape(X.shape[0], X.shape[1], 1, 1)
X_lstm = X.reshape(X.shape[0], X.shape[1], X.shape[2])

# Split into training and testing sets
X_cnn_train, X_cnn_test, X_lstm_train, X_lstm_test, y_train, y_test = train_test_split(
    X_cnn, X_lstm, y, test_size=0.2, random_state=42)

In [ ]:
# Define input shape (modify based on your actual data)
input_shape = (32, 30, 260)

# CNN Model 1
input_cnn = Input(shape=input_shape)

x = Conv1D(filters=32, kernel_size=11, activation='relu')(input_cnn)
x = BatchNormalization()(x)
x = MaxPooling1D(pool_size=2)(x)
x = Dropout(0.5)(x)

x = Conv1D(filters=64, kernel_size=9, activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling1D(pool_size=2)(x)
x = Dropout(0.5)(x)

x = Conv1D(filters=128, kernel_size=5, activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling1D(pool_size=2)(x)
x = Dropout(0.5)(x)

x = Flatten()(x)

# Fully Connected Layers
x = Dense(32, activation='relu')(x)
x = Dense(16, activation='relu')(x)

# LSTM Model
input_lstm = Input(shape=(input_shape[0], input_shape[1] * input_shape[2]))

y = LSTM(units=64, return_sequences=True)(input_lstm)
y = Dropout(0.25)(y)
y = LSTM(units=32, return_sequences=True)(y)
y = Dropout(0.25)(y)
y = LSTM(units=32)(y)
y = Dropout(0.25)(y)

# Combine CNN and LSTM
combined = tf.keras.layers.concatenate([x, y])

# Output Layer
output = Dense(1, activation='sigmoid')(combined)

# Model Definition
model = Model(inputs=[input_cnn, input_lstm], outputs=output)

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='binary_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()